In [1]:
import sys

sys.path.append("../")
from kai.prompt_builder import build_prompt, global_env, Section
import os
import yaml
import tomllib

from kai.model_provider import IBMGraniteModel, IBMOpenSourceModel, OpenAIModel
from kai.kai_logging import KAI_LOG

KAI_LOG.setLevel("info".upper())

In [2]:
# NOTE: You most likely won't have to modify this block, only the last chunk

from kai import Report

report = Report("./analysis/coolstore/output.yaml")

incidents_dict = report.get_impacted_files()

with open(os.path.join("../kai/config.toml"), "rb") as f:
    config = tomllib.load(f)


def add_to_env_force(yaml_str: str):
    documents = yaml.safe_load_all(yaml_str)
    for doc in documents:
        section = Section.from_dict(doc)
        global_env.inner[section.uuid] = section


def build_file_vars(filename, incidents):
    actual_filename = os.path.join("coolstore", filename)

    with open(actual_filename, "r") as f:
        file_contents = f.read()

    for count, incident in enumerate(incidents, 1):
        incident["issue_number"] = count
        incident["analysis_message"] = incident["message"]
        incident["analysis_line_number"] = incident["lineNumber"]

    return {
        "src_file_name": filename,
        "src_file_language": "java",
        "src_file_contents": file_contents,
        "incidents": incidents,
    }


if config["models"]["provider"].lower() == "IBMGranite".lower():
    model_provider = IBMGraniteModel(**config["models"]["args"])
elif config["models"]["provider"].lower() == "IBMOpenSource".lower():
    model_provider = IBMOpenSourceModel(**config["models"]["args"])
elif config["models"]["provider"].lower() == "OpenAI".lower():
    model_provider = OpenAIModel(**config["models"]["args"])
else:
    raise Exception(f"Unrecognized model '{config['models']['provider']}'")


def create_custom_prompt(
    preamble_template: str,
    input_file_template: str,
    output_instructions_template: str,
    pb_vars: dict,
):
    preamble_template = "\n".join(
        "  " + line for line in preamble_template.splitlines()
    )
    input_file_template = "\n".join(
        "  " + line for line in input_file_template.splitlines()
    )
    output_instructions_template = "\n".join(
        "  " + line for line in output_instructions_template.splitlines()
    )

    section_yaml = f"""uuid: $mf_preamble
template: |+
{preamble_template}
---
uuid: $mf_input_file
template: |+
{input_file_template}
---
uuid: $mf_output_instructions
template: |+
{output_instructions_template}
"""

    add_to_env_force(section_yaml)

    return build_prompt("$mf_ibm_model", pb_vars)


def write_to_disk(dir_path, file_path, contents):
    intended_file_path = f"{dir_path}/{file_path}"
    if not os.path.exists(os.path.dirname(intended_file_path)):
        os.makedirs(os.path.dirname(intended_file_path), exist_ok=True)
    print(f"Writing to {intended_file_path}")
    try:
        with open(intended_file_path, "w") as f:
            f.write(contents)
    except Exception as e:
        print(f"Failed to write updated_file @ {intended_file_path} with error: {e}")


for x in global_env.inner:
    print(f"{x}: {global_env.inner[x]}")

INFO - 2024-03-29 15:09:37,364 - [report.py:79 -         _read_report() ] - Reading report from ./analysis/coolstore/output.yaml


Symbol(value='concat'): <function pb_concat at 0x1162aa340>
Symbol(value='format'): <function pb_format at 0x1277905e0>
Symbol(value='build'): <function pb_build at 0x127790680>
Symbol(value='one_of'): <function pb_one_of at 0x127790720>
$test: <kai.prompt_builder.Section object at 0x1274bacc0>
$test_0: <kai.prompt_builder.Section object at 0x12732fe00>
$test_1: <kai.prompt_builder.Section object at 0x1273ca6c0>
$test_2: <kai.prompt_builder.Section object at 0x1272c4d40>
$mf_ibm_model: <kai.prompt_builder.Section object at 0x1272f0200>
$mf_preamble: <kai.prompt_builder.Section object at 0x127699640>
$mf_input_information: <kai.prompt_builder.Section object at 0x127699e50>
$mf_input_file: <kai.prompt_builder.Section object at 0x116429e80>
$mf_issue_header: <kai.prompt_builder.Section object at 0x1274f2e40>
$mf_issue: <kai.prompt_builder.Section object at 0x12716da00>
$mf_issue_solved_example_diff: <kai.prompt_builder.Section object at 0x127766a50>
$mf_issue_solved_example_file: <kai.pro

In [3]:
import time

PREAMBLE = """# Java EE to Quarkus Migration

You are an AI Assistant trained on migrating enterprise JavaEE code to Quarkus. I will give you an example of a JavaEE file and you will give me the Quarkus equivalent.

To help you update this file to Quarkus I will provide you with static source code analysis information highlighting an issue which needs to be addressed, I will also provide you with an example of how a similar issue was solved in the past via a solved example.  You can refer to the solved example for a pattern of how to update the input Java EE file to Quarkus.

Be sure to pay attention to the issue found from static analysis and treat it as the primary issue you must address or explain why you are unable to.

Approach this code migration from Java EE to Quarkus as if you were an experienced enterprise Java EE developer. Before attempting to migrate the code to Quarkus, explain each step of your reasoning through what changes are required and why.

Pay attention to changes you make and impacts to external dependencies in the pom.xml as well as changes to imports we need to consider.

As you make changes that impact the pom.xml or imports, be sure you explain what needs to be updated.

After you have shared your step by step thinking, provide a full output of the updated file.

"""

INPUT_FILE = """## Input File

File name: "{src_file_name}"
Source file contents:
```{src_file_language}
{src_file_contents}
```

"""

OUTPUT_INSTRUCTIONS = """# Output Instructions

Structure your output in Markdown format such as:

## Reasoning

Write the step by step reasoning in this markdown section. If you are unsure of a step or reasoning, clearly state you are unsure and why.

## Updated File

```{src_file_language}
// Write the updated file for Quarkus in this section
```"""

output_dir_path = "./debug"

# Modify this to whatever you want it to be
file_path = "src/main/java/com/redhat/coolstore/model/Order.java"

prompt = create_custom_prompt(
    PREAMBLE,
    INPUT_FILE,
    OUTPUT_INSTRUCTIONS,
    build_file_vars(file_path, incidents_dict[file_path]),
)

prompt_file_path = f"{file_path}.prompt"
write_to_disk(output_dir_path, prompt_file_path, prompt)

# # Comment this out if you don't want to send to llm
start = time.time()
result = model_provider.invoke(prompt)
end = time.time()
content = result.content
print("--- content ---")
print(content)
content_file_path = f"{file_path}.result"
write_to_disk(output_dir_path, content_file_path, content)

print(
    f"<{model_provider.get_current_model_id()}>: Request for {file_path} took {end - start} seconds"
)
print(f"Prompt file written to {output_dir_path}/{prompt_file_path}")
print(f"Result written to {output_dir_path}/{file_path}")

Writing to ./debug/pom.xml.prompt
--- content ---

## Reasoning

1. The first issue is about updating the groupId of the `javaee-web-api` dependency from `javax` to `jakarta.platform`. This is because the `javax` groupId is deprecated and it is recommended to use `jakarta.platform` instead.

2. The second and third issues are similar to the first one. They require updating the groupId of the `javaee-api` and `jboss-jms-api_2.0_spec` dependencies from `javax` to `jakarta.platform`.

3. The fourth and fifth issues are about updating the artifactId of the `javaee-api` and `javaee-web-api` dependencies to `jakarta.jakartaee-api` and `jakarta.jakartaee-web-api` respectively. This is because the old artifactIds are deprecated and it is recommended to use the new ones instead.

4. The sixth issue is about replacing the JavaEE/Jakarta JMS dependency with Smallrye Reactive. This is because Quarkus does not support JMS and it is recommended to use Smallrye Reactive instead.

5. The seventh issue